# Bayesian Optimization: scalable edition

In [ ]:
def get_free_gpu():
    from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo, nvmlDeviceGetCount
    nvmlInit()

    return np.argmax([
        nvmlDeviceGetMemoryInfo(nvmlDeviceGetHandleByIndex(i)).free
        for i in range(nvmlDeviceGetCount())
    ])

In [ ]:
import numpy as np
import torch

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

if torch.cuda.is_available():
    cuda_id = get_free_gpu()
    DEVICE = 'cuda:%d' % (get_free_gpu(), )
    print('Selected %s' % (DEVICE, ))
else:
    DEVICE = 'cpu'
    print('WARNING: using cpu!')

### please, don't remove the following line
x = torch.tensor([1], dtype=torch.float32).to(DEVICE)

In [ ]:
def plot_gp(gp, X=None, y=None, objective=None):
    with torch.no_grad():
        xs_ = np.linspace(0, 1, 100)
        mean, std =  gp(
            torch.tensor(xs_.reshape(-1, 1), device=DEVICE, dtype=torch.float32)
        )

    mean_ = mean.cpu().numpy()
    std_ = std.cpu().numpy()
    
    if X is not None and y is not None:
        plt.scatter(X, y, color=plt.cm.tab10(0), s=100)
    
    if objective is not None:
        plt.plot(xs_, objective(xs_), '--', color='black')
    
    plt.plot(xs_, mean_, lw=2)
    plt.fill_between(
        xs_, mean_ - std_, mean_ + std_,
        alpha=0.2,
        color=plt.cm.tab10(0)
    )

## A very special case of GP --- linear non-kernel version of GP.

In [ ]:
class LinGP(torch.nn.Module):
  def __init__(self, sigma_w=1, sigma_b=1, sigma_n=0.5):
    """
    Linear Gaussian Process:
        y ~ N(W X + b, (sigma_n) ** 2)
        W_i ~ N(0, (sigma_w) ** 2)
        b ~ N(0, (sigma_b) ** 2)

    X, y = observed values, X - tensor of size (N, d), y - tensor of size (N, ).
    sigma_w - variance of the prior over W.
    sigma_b - variance of the prior over b.
    sigma_n - variance of the noise.


    Similar to GP with sklearn.gaussian_process.kernels.DotProduct with:
        sigma_b0 = sigma_b;
        sigma_0_bounds=(sigma_b, sigma_b),
        alpha (GP parameter) = sigma_n ** 2

    DotProduct kernel assumes sigma_w = 1.
    """
    super(LinGP, self).__init__()

    self.sigma_w = sigma_w
    self.sigma_b = sigma_b
    self.sigma_n = sigma_n

    self.M = torch.zeros(0, 0, dtype=torch.float32)
    self.A_inv_sigma = torch.zeros(0, 0, dtype=torch.float32)

  def fit(self, X, y):
    X = torch.cat([
      torch.ones(
        X.shape[0], 1,
        device=X.device, dtype=X.dtype,
        requires_grad=X.requires_grad
      ) * self.sigma_b / self.sigma_w,
      X
    ], dim=1)

    cov_w = torch.eye(
      n=X.shape[1],
      device=X.device, dtype=X.dtype,
      requires_grad=False
    ) * ((self.sigma_n / self.sigma_w) ** 2)

    A = torch.matmul(X.t(), X) + cov_w

    A_inv = torch.inverse(A)

    self.M = torch.matmul(
      y,
      torch.matmul(X, A_inv)
    )

    self.A_inv_sigma = A_inv * (self.sigma_n ** 2)

  def forward(self, x):
    x = torch.cat([
      torch.ones(
        x.shape[0], 1,
        device=x.device, dtype=x.dtype,
        requires_grad=x.requires_grad
      ) * self.sigma_b / self.sigma_w,
      x
    ], dim=1)

    mean = torch.sum(x * self.M[None, :], dim=(1,))
    sigma_sqr = torch.sum(
      torch.matmul(x, self.A_inv_sigma) * x,
      dim=(1,)
    )

    return mean, torch.sqrt(sigma_sqr)

In [ ]:
def simple_objective(x):
    return np.exp(-2 * x + 1) + np.exp(x) - 2.5

X = np.array([0.1, 0.4, 0.9])
y = simple_objective(X)

In [ ]:
N_BASIS = 7
def basis_expansion(x):
    return np.stack([
        x ** i
        for i in range(N_BASIS)
    ], axis=1)

In [ ]:
lin_gp = LinGP(sigma_n=1e-1, sigma_b=1).to(DEVICE)
lin_gp.fit(
    torch.tensor(basis_expansion(X), dtype=torch.float32, device=DEVICE),
    torch.tensor(y.reshape(-1), dtype=torch.float32, device=DEVICE),
)

In [ ]:
def predict_lin_gp(X):
    with torch.no_grad():
        X = torch.cat([
            X ** i
            for i in range(N_BASIS)
        ], dim=1)
        return lin_gp(X)
    
plot_gp(predict_lin_gp, X, y, objective=simple_objective)

## Let's compare it to pyro

In [ ]:
import pyro
pyro.set_rng_seed(111222333)

from pyro.contrib.gp.models import GPRegression
from pyro.contrib.gp.kernels import Linear

In [ ]:
X_large = torch.randn(1024, 32, dtype=torch.float32, device=DEVICE)
y_large = torch.randn(1024, dtype=torch.float32, device=DEVICE)

In [ ]:
%%time

lin_gp.fit(X_large, y_large)

In [ ]:
%%time

pyro.clear_param_store()
gp = GPRegression(
    X_large, y_large,

    noise=torch.tensor(1e-2),
    kernel=Linear(
        input_dim=1,
        variance=torch.tensor(1, dtype=torch.float32, device=DEVICE),
    ),
).to(DEVICE)

with torch.no_grad():
    _ = gp(X_large)

## Network with multiple head

In [ ]:
class NeuralGP(torch.nn.Module):
    def __init__(self, n_inputs, n, lin_gp):
        super(NeuralGP, self).__init__()

        self._basis = [
            torch.nn.Linear(n_inputs, 4 * n), torch.nn.ELU(),
            torch.nn.Linear(4 * n, 2 * n), torch.nn.ELU(),
            torch.nn.Linear(2 * n, n), torch.nn.ELU(),
        ]
        
        for i, f in enumerate(self._basis):
            self.add_module('f%d' % (i, ), f)
        
        self.lin_reg = torch.nn.Linear(n, 1)
        self.lin_gp = lin_gp
        
        self.dropout = torch.nn.Dropout(p=0.1)
        
        self.opt = torch.optim.Adam(
            lr=2e-3, weight_decay=1e-3,
            params=self.parameters()
        )
    
    def basis(self, x, dropout=False):
        for f in self._basis:
            x = f(x)
            
            if dropout:
                x = self.dropout(x)
        
        return x
    
    def forward(self, x, dropout=False):
        basis = self.basis(x, dropout=dropout)
        return self.lin_reg(basis)
    
    def fit(self, X, y, n_iters=1024, progress_bar=lambda x: x):
        ### training regression NN
        
        X = torch.tensor(X, dtype=torch.float32, device=DEVICE)
        y = torch.tensor(y, dtype=torch.float32, device=DEVICE)
        
        losses = np.zeros(n_iters)
    
        for i in progress_bar(range(n_iters)):
            with torch.no_grad():
                indx = torch.randint(low=0, high=X.shape[0], size=(32, ), device=DEVICE)
                X_batch = X[indx]
                y_batch = y[indx]

            self.opt.zero_grad()
            predictions = self.lin_reg(
                self.basis(X_batch, dropout=True)
            ).view(-1)

            loss = torch.mean((predictions - y_batch) ** 2)

            loss.backward()
            self.opt.step()

            losses[i] = loss.item()
        
        ### fitting linGP
        with torch.no_grad():
            basis = self.basis(X)
            self.lin_gp.fit(basis, y)
        
        return losses
    
    def get_predictions(self, X):
        with torch.no_grad():
            X = torch.tensor(X, dtype=torch.float32, device=DEVICE)
            return self.lin_reg(self.basis(X)).view(-1).cpu().numpy()
    
    def get_basis(self, X):
        with torch.no_grad():
            X = torch.tensor(X, dtype=torch.float32, device=DEVICE)
            return self.basis(X).cpu().numpy()
    
    def gp(self, X):
        basis = self.basis(X)
        return self.lin_gp(basis)

## Simple objective

In [ ]:
lin_gp = LinGP(sigma_n=1e-2, sigma_b=1).to(DEVICE)
nngp = NeuralGP(n_inputs=1, n=8, lin_gp=lin_gp).to(DEVICE)

In [ ]:
X = np.array([0.1, 0.4, 0.9])
y = simple_objective(X)

nngp.fit(X.reshape(-1, 1), y, n_iters=1024, progress_bar=tqdm) 

In [ ]:
plot_gp(nngp.gp, X, y, objective=simple_objective)

xs = np.linspace(0, 1, num=100)
p = nngp.get_predictions(xs.reshape(-1, 1))
plt.plot(xs, p)

## Multi-modal objective

In [ ]:
m = 5
w = np.random.uniform(-1, 1, size=m)
periods = np.arange(1, m + 1) * np.pi

def objective_many_minima_much_challenge(x):
    return np.sum(
        w[:, None] * np.cos(x[None, :] * periods[:, None]),
        axis=0
    ) + x ** 2

xs = np.linspace(0, 1, num=100)
plt.plot(xs, objective_many_minima_much_challenge(xs))

In [ ]:
X = np.concatenate([
    np.random.uniform(0, 0.45, size=10),
    np.random.uniform(0.55, 1, size=10),
])
y = objective_many_minima_much_challenge(X)

In [ ]:
lin_gp = LinGP(sigma_n=1e-2, sigma_b=1).to(DEVICE)
nngp = NeuralGP(n_inputs=1, n=32, lin_gp=lin_gp).to(DEVICE)

In [ ]:
nngp.fit(X.reshape(-1, 1), y, n_iters=8 * 1024, progress_bar=tqdm)

In [ ]:
plot_gp(nngp.gp, X, y, objective=objective_many_minima_much_challenge)

xs = np.linspace(0, 1, num=100)
basis = nngp.get_basis(xs.reshape(-1, 1))

for i in range(basis.shape[1]):
    plt.plot(xs, basis[:, i], alpha=0.1, color=plt.cm.tab10(0))

plt.plot(xs, nngp.get_predictions(xs.reshape(-1, 1)), lw=2)

### Utils

*stolen from the previous seminar*

In [ ]:
def minimize_acq(x0, gp, acq, lr=1e-1, n_iters=128, progress_bar=lambda x: x):
    x = torch.tensor(x0, dtype=torch.float32, device=DEVICE, requires_grad=True)
    
    ### not stochastic in this case
    opt = torch.optim.SGD(lr=lr, params=[x])
    
    for _ in progress_bar(range(n_iters)):
        opt.zero_grad()        
        torch.sum(acq(gp, x)).backward()
        opt.step()
        
        with torch.no_grad():
            x = torch.clamp(x, 0, 1)

    with torch.no_grad():
        values = acq(gp, x)

    return x.detach().cpu().numpy(), values.detach().cpu().numpy()

## Endurance test

In [ ]:
def objective(x):
    return np.sum(x ** 2)

In [ ]:
def lcb(gp, x, alpha=1.0):
    mean, std = gp(x)
    
    return mean - alpha * std

### Purely explorative acquisition function.
def exlorative(gp, x, alpha=1.0):
    mean, std = gp(x)
    return - std

### Purely exploitative acquisition function.
def exploitative(gp, x, alpha=1.0):
    mean, std = gp(x)
    
    return mean

In [ ]:
def cummin(fs):
    result = np.zeros_like(fs)
    
    result[0] = fs[0]
    
    for i in range(1, fs.shape[0]):
        if result[i - 1] < fs[i]:
            result[i] = result[i - 1]
        else:
            result[i] = fs[i]
    
    return result


In [ ]:
def plot_convergence(y, color=plt.cm.tab10(0), label=None):
    y = np.array(y)
    iters = np.arange(len(y))
    
    plt.plot(iters, cummin(y), color=color, label=label)
    plt.scatter(iters, y, alpha=0.5, color=color, label=label)

In [ ]:
def plot(f, history=None, trajectory=False):
    xs = np.linspace(0, 1, num=250)
    X, Y = np.meshgrid(xs, xs)

    Z = np.stack([X.reshape(-1), Y.reshape(-1)], axis=1)
    F = f(Z[:, 0], Z[:, 1]).reshape(xs.shape[0], xs.shape[0])
    
    plt.figure(figsize=(6, 6))
    plt.ylim([0, 1])
    plt.xlim([0, 1])

    plt.contour(
        xs, xs, F,
        levels=np.linspace(0, 4, num=20),
        colors=[plt.cm.tab10(0)]
    )
    plt.scatter([1], [1], marker='*', s=500, color=plt.cm.tab10(3), zorder=5)
    
    if history is not None:
        plt.scatter(history[:, 0], history[:, 1], color=plt.cm.tab10(1), s=100)
        if trajectory:
            plt.plot(history[:, 0], history[:, 1], color=plt.cm.tab10(1), lw=3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
def log_resenbrock(x1, x2):
    x1 = x1 * 3 - 0.5
    x2 = x2 * 3 - 0.5
    
    return np.log1p(
        (1 - x1) ** 2 + 1 * (x2 - x1 ** 2) ** 2
    )

## Random Search

*(as a baseline)*

In [ ]:
X_rs = np.random.uniform(size=(32, 2))
y_rs = log_resenbrock(X_rs[:, 0], X_rs[:, 1])

## NN-BO

In [ ]:
def nnbo(objective, nngp, n_iters=32):
    X = list()
    y = list()

    for _ in tqdm(range(n_iters)):
        if len(X) < 5:
            suggestion = np.random.uniform(size=2)
        else:
            suggestions, values = minimize_acq(
                np.random.uniform(size=(32, 2)),
                nngp.gp,
                lcb
            )
            best = np.argmin(values)
            suggestion = suggestions[best]
        
        value = objective(*suggestion)

        X.append(suggestion)
        y.append(value)

        nngp.fit(np.array(X), np.array(y), n_iters=1024)
    
    return np.array(X), np.array(y)

## Rosenbrock test

In [ ]:
lin_gp = LinGP(sigma_n=1, sigma_b=1).to(DEVICE)
nngp = NeuralGP(n_inputs=2, n=16, lin_gp=lin_gp).to(DEVICE)

In [ ]:
X, y = nnbo(log_resenbrock, nngp, n_iters=32)

In [ ]:
plot(log_resenbrock, history=X)

In [ ]:
plot_convergence(y, label='NN-BO', color=plt.cm.tab10(0))
plot_convergence(y_rs, label='RS', color=plt.cm.tab10(1))
plt.yscale('log')

## Trial by ~~fire~~ HIGGS

In [ ]:
data = np.load('../../../share/HIGGS-small.npy')

In [ ]:
from sklearn.model_selection import train_test_split

X_all, y_all = data[:, 1:], data[:, 0]

### casting labels to int
y_all = np.where(y_all > 0.5, np.int(1), np.int(0))

X_train, X_val, y_train, y_val = train_test_split(X_all, y_all, test_size=0.25)

print('Train     :', X_train.shape, y_train.shape)
print('Validation:', X_val.shape, y_val.shape)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

def get_gb(n_trees, log_learning_rate):
    n_trees = int(100 * n_trees + 1)
    log_learning_rate = log_learning_rate * 6 - 6
    learning_rate = np.exp(log_learning_rate)

    return GradientBoostingClassifier(
        learning_rate=learning_rate,
        n_estimators=n_trees, max_depth=3,
        subsample=0.1, random_state=123
    )

def gb_quality(n_trees, log_learning_rate):
    clf = get_gb(n_trees, log_learning_rate)
    
    predictions = clf.fit(X_train, y_train).predict_proba(X_val)[:, 1]
    
    error = 1 - roc_auc_score(y_val, predictions)
    computational_penalty = 1e-1 * n_trees
    
    return error + computational_penalty

In [ ]:
lin_gp = LinGP(sigma_n=1e-1, sigma_b=1).to(DEVICE)
nngp = NeuralGP(n_inputs=2, n=16, lin_gp=lin_gp).to(DEVICE)

In [ ]:
X, y = nnbo(gb_quality, nngp, n_iters=32)

In [ ]:
### looking for the best guess with an exploitative acqusition function
suggestions, values = minimize_acq(
    x0=np.random.uniform(0, 1, size=(128, 2)),
    gp=nngp.gp,
    n_iters=1024,
    progress_bar=tqdm,
    acq=exploitative
)
best = np.argmin(values)
best_guess = suggestions[best]

In [ ]:
clf = get_gb(*best_guess)
    
predictions = clf.fit(X_train, y_train).predict_proba(X_val)[:, 1]

auc = roc_auc_score(y_val, predictions)
computational_penalty = 1e-1 * x[0]

print('ROC AUC: %.3lf' % (auc, ))
print('complexity: %.3lf' % (computational_penalty, ))
print('objecive: %.3lf' % (1 - auc + computational_penalty))

assert True

In [ ]:
plot_convergence(y)